Ohje käyttäjälle:

- Aja ensin kaikki työkirjan solut
- Kartan yläpuolella olevasta valikosta voit suodattaa hakutuloksia:
    - Osoite: Anna osoite, jonka ympäristöstä haluat nähdä asunnot
    - Kaupunki: Kaupunki, jossa annettu osoite sijaitsee
    - Etäisyys: Syötä etsittyjen asuntojen maksimietäisyys kilometreinä annetusta osoitteesta. Käytä tarvittaessa
      desimaalierottimena pistettä.
    - Asuntotyyppi: Valitse alasvetovalikosta haluamasi asuntotyyppi
- Klikkaamalla kartalla näkyvää paikkamerkkiä näet tietoja asunnosta
- Voit halutettasi syöttää vain kaupungin

Raportti sisältää Tampereen, Turun, Jyväskylän ja Oulun omakoti- ja erillistalojen tiedot, jotka on ilmoitettu etuovi.com:ssa 
riittävän kattavin tiedoin

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import haversine as hs
from haversine import Unit
import folium
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim


In [6]:
# read data
d = open('preprosessed_data.json')
data = json.load(d)
d.close()
df = pd.DataFrame(data)
df=df.dropna()

geolocator = Nominatim(user_agent = "joda ht")

In [29]:
@interact
def show_houses(Osoite = 'Keskustori 1', Kaupunki = 'Tampere', Etäisyys="20", \
                Asuntotyyppi = ['Kaikki','Omakotitalo', 'Erillistalo']):
    d={'Address': [Osoite], 'City':[Kaupunki]}
    dist = pd.DataFrame(data=d)

    dist['Address_geo'] = dist['Address'] + ", " + dist['City'] + ", Finland"

    dist['Address_loc'] = dist['Address_geo'].apply(geolocator.geocode)
    # select koordinates from loc
    dist['Address_point'] = dist['Address_loc'].apply(lambda loc: tuple(loc.point) if loc else None)
    #split coordinates to latitude, longitude and altitude
    dist[['lat', 'lon', 'altitude']] = pd.DataFrame(dist['Address_point'].to_list(), index=dist.index)
    center = (dist['lat'][0], dist['lon'][0])

    #Create the map
    my_map = folium.Map(location = center, zoom_start = 10)

    # Add addresses to the map
    for index, row in df.iterrows():
        if Asuntotyyppi == 'Kaikki':
            if hs.haversine(center,(row['lat'],row['lon']))<float(Etäisyys):
                if row.point != None:
                    text =str(row.house_type) + " Hinta: " + str(row.price) + " €" + "\n Koko: " + str(row['size']) + " m2 " \
                    +" Huonejako: " + str(row.rooms) + " Osoite: " + row.address
                    folium.Marker([row.lat,row.lon], popup = text).add_to(my_map)
        else:
            if hs.haversine(center,(row['lat'],row['lon']))<float(Etäisyys) and row['house_type'] == Asuntotyyppi:
                if row.point != None:
                    text =str(row.house_type) + " Hinta: " + str(row.price) + " €" + "\n Koko: " + str(row['size']) + " m2 " \
                    +" Huonejako: " + str(row.rooms) + " Osoite: " + row.address
                    folium.Marker([row.lat,row.lon], popup = text).add_to(my_map)
    
    return my_map

interactive(children=(Text(value='Keskustori 1', description='Osoite'), Text(value='Tampere', description='Kau…